In [ ]:
%matplotlib inline

# Extrapolation method for stress result of a 3D element {#extrapolation_test_stress_3Delement}

This example shows how to compute the nodal components stress from
Gaussian points (integration points) for a 3D element by using of
extrapolation.

Extrapolate results available at Gaussian or quadrature points to nodal
points for a field or fields container. The available elements are:

-   Linear quadrangle
-   Parabolic quadrangle
-   Linear hexagonal
-   Quadratic hexagonal
-   Linear tetrahedral
-   Quadratic tetrahedral

Here are the steps for extrapolation:

1.  Get the data source\'s solution from the integration points. (This
    result file was generated with the Ansys Mechanical APDL (MAPDL)
    option `EREXS, NO`).
2.  Use the extrapolation operator to compute the nodal stress.
3.  Get the result for nodal stress from the data source. The analysis
    was computed by MAPDL.
4.  Compare the result for nodal stress from the data source and the
    nodal stress computed by the extrapolation method.


In [ ]:
from ansys.dpf import core as dpf
from ansys.dpf.core import examples

Get the data source\'s analysis of integration points and analysis
reference


In [ ]:
datafile = examples.download_extrapolation_3d_result()

# Get integration points (Gaussian points)
data_integration_points = datafile["file_integrated"]
data_sources_integration_points = dpf.DataSources(data_integration_points)

# Get the reference
dataSourceref = datafile["file_ref"]
data_sources_ref = dpf.DataSources(dataSourceref)

# Get the mesh
model = dpf.Model(data_integration_points)
mesh = model.metadata.meshed_region

# Operator instantiation scoping
op_scoping = dpf.operators.scoping.split_on_property_type()  # operator instantiation
op_scoping.inputs.mesh.connect(mesh)
op_scoping.inputs.requested_location.connect("Elemental")
mesh_scoping = op_scoping.outputs.mesh_scoping()

# Extrapolate from integration points for stress result

This example uses the `gauss_to_node_fc` operator to compute the nodal
component stress result from the stress result of integration points.


In [ ]:
# Create stress operator to get stress result of integration points
stressop = dpf.operators.result.stress()
stressop.inputs.data_sources.connect(data_sources_integration_points)
stress = stressop.outputs.fields_container()

# Nodal stress result of integration points

> The MAPLD command `ERESX,NO` is used to copy directly the Gaussian
> (integration) points results to the nodes, instead of the results at
> nodes or elements (which are interpolation of results at a few gauss
> points). The following plot shows the nodal values, which are the
> averaged values of stresses at each node. The value shown at the node
> is the average of the stresses from the Gaussian points of each
> element that it belongs to.


In [ ]:
# Plot
stress_nodal_op = dpf.operators.averaging.elemental_nodal_to_nodal_fc()
stress_nodal_op.inputs.fields_container.connect(stress)
mesh.plot(stress_nodal_op.outputs.fields_container())

Create operator `gauss_to_node_fc` and compute nodal component stress by
applying the extrapolation method.


In [ ]:
ex_stress = dpf.operators.averaging.gauss_to_node_fc()
# connect mesh
ex_stress.inputs.mesh.connect(mesh)
# connect fields container stress
ex_stress.inputs.fields_container.connect(stress)
# get output
fex = ex_stress.outputs.fields_container()

# Stress result of reference Ansys Workbench


In [ ]:
# Stress from file dataSourceref
stressop_ref = dpf.operators.result.stress()
stressop_ref.inputs.data_sources.connect(data_sources_ref)
stressop_ref.inputs.mesh_scoping.connect(mesh_scoping)
stress_ref = stressop_ref.outputs.fields_container()

# Plot

Show plots of the extrapolation\'s stress result and the reference\'s
stress result


In [ ]:
# extrapolation
fex_nodal_op = dpf.operators.averaging.elemental_nodal_to_nodal_fc()
fex_nodal_op.inputs.fields_container.connect(fex)
mesh.plot(fex_nodal_op.outputs.fields_container())
# reference
stress_ref_nodal_op = dpf.operators.averaging.elemental_nodal_to_nodal_fc()
stress_ref_nodal_op.inputs.fields_container.connect(stress_ref)
mesh.plot(stress_ref_nodal_op.outputs.fields_container())

# Compare stress results

Compare the stress result computed by extrapolation and the reference\'s
result. Check if the two fields container are identical. The maximum
tolerance gap between two compared values is 1e-2. The smallest value
that is considered during the comparison step: all the `abs(values)` in
field less than 1e-8 is considered as null.


In [ ]:
# operator AreFieldsIdentical_fc
op = dpf.operators.logic.identical_fc()
op.inputs.fields_containerA.connect(fex)
op.inputs.fields_containerB.connect(stress_ref)
op.inputs.tolerance.connect(1.0e-8)
op.inputs.small_value.connect(0.01)
op.outputs.boolean()